In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import napari
import colorcet as cc
import pandas as pd

import dnt

spots_directory = Path(r"D:\Tracking\DrosophilaNucleusTracking\data\spots")
save_path = Path(r"D:\Tracking\DrosophilaNucleusTracking\animations\output\wave_animations")

include = [1, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14]
condition = [0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2]
earliest_frames = [25, 43, 63, 80, 55, 150, 23, 36, 17, 50, 3]
# include = [9]
# earliest_frames = [55]

dnt.set_plot_style()

spots_dfs, metadatas, stems = dnt.load_spots_data(spots_directory, include)

df = spots_dfs[0]
cycles = [10, 11, 12, 13, 14]

print(df.columns)

['lightsheet_20241104_spots', 'lightsheet_20250131_spots', 'lightsheet_20250206_spots', 'lightsheet_20250414_spots', 'lightsheet_20250625_spots', 'lightsheet_20250627_spots', 'lightsheet_20250702_spots', 'lightsheet_20250704_spots', 'lightsheet_20250705_spots', 'lightsheet_bcd_20251105a_spots', 'lightsheet_bcd_20251106_spots', 'lightsheet_trk_20250318_spots', 'lightsheet_trk_20250319a_spots', 'lightsheet_trk_20250319b_spots', 'lightsheet_trk_20250321_spots', 'lightsheet_trk_20250325_spots']


reading spots dfs: 0it [00:00, ?it/s]

[  0  35  60 110 160 280] 25 246


reading spots dfs: 2it [00:01,  1.61it/s]

[  0  55  90 140 200 350] 0 332


...

[  0  25  55 100 180 280] 0 241


reading spots dfs: 14it [00:14,  1.29s/it]

[  0  55  90 140 200 350] 0 311


reading spots dfs: 16it [00:16,  1.05s/it]

Index(['time_since_nc11', 'z', 'y', 'x', 'AP', 'theta', 'area', 'radius',
       'intensity_mean', 'cycle', 'display_z', 'display_y', 'display_x',
       'px_z', 'px_y', 'px_x', 'px_area', 'uv_v', 'uv_u', 'uv_z',
       'uv_distance_from_edge', 'distance_from_surface', 'uv_area',
       'area_distortion', 'mesh_name', 'interpolated', 'track_id',
       'tracklet_id', 'parent_id', 'n_children', 'n_parents', 'AP_raw',
       'AP_um', 'AP_um_centered', 'um_from_anterior', 'um_from_posterior',
       'distance_to_pole', 'time', 'trk', 'problematic', 'frame',
       'trackmate_id', 'status', 'source', 'is_parent', 'is_child',
       'pseudotime', 'distance', 'cycle_pseudotime', 'dx', 'dy', 'dz', 'dAP',
       'dtheta', 'dtot', 'dAP_abs'],
      dtype='object')


In [14]:
# def interpolate_points(df, factors, extent=9):
#     points = df[["frame", 'z', 'y', 'x']].values
#
#     parents = df["parent_id"].values
#     mask = parents != -1
#     parents = parents[mask].astype(int)
#     parent_points = df.loc[parents, ["frame", 'z', 'y', 'x']].values
#     child_points = points[mask]
#
#     offsets = np.linspace(0, 1, extent+2)[1:]
#
#     new_points = parent_points[:, None, :] + (child_points - parent_points)[:, None, :] * offsets[None, :, None]
#     new_points = new_points.reshape(-1, 4)
#     new_points[:, 0] *= extent + 1  # Adjust frame numbers
#
#     new_factors = {}
#     for factor in factors:
#         new_factors[factor] = np.repeat(df.loc[mask, factor].values, extent + 1)
#
#     return new_points, new_factors
#
# new_points, new_factors = interpolate_points(spots_dfs[0], factors=["cycle", "radius"])

In [4]:
import napari
from napari.utils.theme import get_theme, register_theme

bg_color = "#000000"

custom_theme = get_theme("light")
print(custom_theme)
custom_theme.canvas = bg_color
register_theme("custom_light", custom_theme, "custom")



k = 8
df = spots_dfs[k]
df = df[df["frame"] >= earliest_frames[k]]
df = df[df["cycle"] > 9]

# map na radii to frame average radius
mean_radius_per_frame = df.groupby("frame")["radius"].mean()
na_radius_mask = df["radius"].isna()
df.loc[na_radius_mask, "radius"] = df.loc[na_radius_mask, "frame"].map(mean_radius_per_frame)

print(df.columns)

points = df[['frame', 'z', 'y', 'x']].values
colors = [dnt.palettes.nc[i] for i in df["cycle"]]

viewer = napari.Viewer(ndisplay=3)
viewer.theme = "custom_light"
viewer.add_points(points, size=df["radius"]*2, face_color=colors, border_color='black', border_width=0.1)
napari.run()


id='light' label='Default Light' syntax_style='default' canvas=Color('white', rgb=(255, 255, 255)) console=Color('white', rgb=(255, 255, 255)) background=Color('#efebe9', rgb=(239, 235, 233)) foreground=Color('#d6d0ce', rgb=(214, 208, 206)) primary=Color('#bcb8b5', rgb=(188, 184, 181)) secondary=Color('#969290', rgb=(150, 146, 144)) highlight=Color('#a39e9c', rgb=(163, 158, 156)) text=Color('#3b3a39', rgb=(59, 58, 57)) icon=Color('#6b6967', rgb=(107, 105, 103)) warning=Color('#e3b617', rgb=(227, 182, 23)) error=Color('#ff121f', rgb=(255, 18, 31)) current=Color('#fdf094', rgb=(253, 240, 148)) font_size='9pt'
Index(['time_since_nc11', 'z', 'y', 'x', 'AP', 'theta', 'area', 'radius',
       'intensity_mean', 'cycle', 'display_z', 'display_y', 'display_x',
       'px_z', 'px_y', 'px_x', 'px_area', 'uv_v', 'uv_u', 'uv_z',
       'uv_distance_from_edge', 'distance_from_surface', 'uv_area',
       'area_distortion', 'mesh_name', 'interpolated', 'track_id',
       'tracklet_id', 'parent_id', 'n

In [5]:
from tqdm import tqdm

stem = stems[k]

animation_save_path = save_path / stem
animation_save_path.mkdir(parents=True, exist_ok=True)

for i, frame in tqdm(enumerate(df["frame"].unique())):
    viewer.dims.set_current_step(0, i)
    save_filepath = animation_save_path / f"{stem}_frame_{frame:03d}.tif"
    viewer.screenshot(save_filepath, flash=False)


264it [00:16, 15.61it/s]
